In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.ticker as ticker
import IPython.display

plt.rcParams['font.family'] = ["D2Coding"]

NOTEBOOK_ID = "WAVE-EQUATION"
OUTPUT_PATH = f"out/{NOTEBOOK_ID}/"

if not os.path.isdir(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)


# Wave Equation

위치 $\mathbf{x}$, 시각 $t$에서의 진폭을 나타내는 $u(\mathbf{x}, t)$에 대한 선형 쌍곡 편미분 방정식으로 다음과 같다.

$$\frac{\partial^2 u}{\partial t^2} = c^2 \nabla^2 u$$

이때, $c$는 파동의 속도.



## Wave Equation 1D

$$\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2}$$

In [2]:
t, dt = np.linspace(0, 10, 100000, retstep=True)
x, dx = np.linspace(-4, 4, 100, retstep=True)
nt = len(t)
nx = len(x)

w = np.zeros((nt, nx))
w[0] = w[1] = np.zeros_like(x)

c = 0.7

for _t in range(2, nt):
    for _x in range(1, nx - 1):
        w[_t, _x] = 2 * w[_t - 1, _x] - w[_t - 2, _x] + c ** 2 * (dt / dx) ** 2 * (
                w[_t - 1, _x + 1] - 2 * w[_t - 1, _x] + w[_t - 1, _x - 1])
    # boundary: reflecting
    w[_t, 0] = 0  #w[_t, 1]
    w[_t, -1] = 0  #w[_t, -2]

    # additional
    w[_t, 25] = 0.125 * np.sin(t[_t] * np.pi)
    w[_t, 90] = 0.125 * np.sin(t[_t] * np.pi)

In [3]:
fig, ax = plt.subplots(figsize=(8, 8))

W1, = plt.plot(x, w[0], color='red')

ax.set_ylim((-1.2, 1.2))


def animate(frame):
    W1.set_data(x, w[frame * 1000, :])
    #ax.set_title(f"t={round(t[frame * 1000], 3)}")
    return W1,


anim = animation.FuncAnimation(fig, animate, frames=100, interval=100)
plt.close()

anim.save(OUTPUT_PATH + "wave1d.mp4", dpi=100)
IPython.display.Video(OUTPUT_PATH + "wave1d.mp4", width=80 * 8, height=80 * 8)


## Wave Equation 2D

$$\frac{\partial^2 u}{\partial t^2} = c^2\left(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right)$$




In [4]:
t, dt = np.linspace(0, 10, 100000, retstep=True)
x, dx = np.linspace(-4, 4, 50, retstep=True)
y, dy = np.linspace(-4, 4, 50, retstep=True)
X, Y = np.meshgrid(x, y)
X = X.T
Y = Y.T

nt = len(t)
nx = len(x)
ny = len(x)

w = np.zeros((nt, nx, ny))
w[0] = w[1] = np.zeros_like(X)

c = 0.7

for _t in range(2, nt):
    for _x in range(1, nx - 1):
        for _y in range(1, ny - 1):
            w[_t, _x, _y] = 2 * w[_t - 1, _x, _y] - w[_t - 2, _x, _y] + c ** 2 * ((dt / dx) ** 2 * (
                    w[_t - 1, _x + 1, _y] - 2 * w[_t - 1, _x, _y] + w[_t - 1, _x - 1, _y]) + (dt / dy) ** 2 * (
                                              w[_t - 1, _x, _y + 1] - 2 * w[_t - 1, _x, _y] + w[_t - 1, _x, _y - 1]))
    # boundary: reflecting
    w[_t, :, 0] = 0
    w[_t, :, -1] = 0
    w[_t, 0, :] = 0
    w[_t, -1, :] = 0

    # additional
    w[_t, 24, 24] = 0.125 * np.sin(t[_t] * np.pi)


In [12]:
fig, ax = plt.subplots(figsize=(6, 6))

lignite_norm = plt.Normalize(-0.05, 0.05)


W1 = ax.imshow(w[0], norm=lignite_norm, cmap=plt.cm.Blues)

fig.colorbar(plt.cm.ScalarMappable(lignite_norm, cmap=plt.cm.Blues))

def animate(frame):
    W1.set_data(w[frame*10])
    fig.canvas.flush_events()
    return W1,

anim = animation.FuncAnimation(fig, animate, frames=10000, interval=1)
plt.close()

anim.save(OUTPUT_PATH + "wave2d.mp4", dpi=100)
IPython.display.Video(OUTPUT_PATH + "wave2d.mp4", width=80 * 8, height=80 * 8)


In [112]:
fig, ax = plt.subplots(subplot_kw={'projection': '3d'}, figsize=(11, 10))

surf = ax.plot_surface(X, Y, w[0], facecolors=plt.cm.BuPu(lignite_norm(w[0])), linewidth=1.5)
surf.set_facecolor("white")

ax.set_zlim((-0.05, 0.05))

def animate(frame):
    ax.clear()
    surf = ax.plot_surface(X, Y, w[frame*1000], facecolors=plt.cm.BuPu(lignite_norm(w[frame*1000])), linewidth=1.5)
    surf.set_facecolor("white")
    
    ax.set_zlim((-0.05, 0.05))

    return surf,

anim = animation.FuncAnimation(fig, animate, frames=100, interval=100)
plt.close()

anim.save(OUTPUT_PATH + "wave2d-3d.mp4", dpi=100)
IPython.display.Video(OUTPUT_PATH + "wave2d-3d.mp4", width=110 * 8, height=100 * 8)

# Wave Equation에서의 Boundary Condition